## Import libraries

In [2]:
from datetime import datetime
import json
from logging import basicConfig, getLogger, StreamHandler, DEBUG, WARNING
import os
import sys
from typing import Any, Dict, List

import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    log_loss,
    precision_score,
    recall_score,
)
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras

sys.path.insert(0, '../')

from src.data_prep.load import load_raw_data
from src.utils import check_class_balance, round
from src.utils import plot_feature_importance, plot_shap_summary, plot_confusion_matrix
from models.stacked_lstm import train_and_predict

In [3]:
CUR_DIR = os.path.dirname(os.path.abspath('__file__'))  # Path to current directory

### Logging settings

In [4]:
EXEC_TIME = "deep-conv-lstm-" + datetime.now().strftime("%Y%m%d-%H%M%S")
LOG_DIR = os.path.join(CUR_DIR, f"../logs/stacked_lstm_logs/{EXEC_TIME}")
os.makedirs(LOG_DIR, exist_ok=True)  # Create log directory

formatter = "%(levelname)s: %(asctime)s: %(filename)s: %(funcName)s: %(message)s"
basicConfig(filename=f"{LOG_DIR}/{EXEC_TIME}.log", level=DEBUG, format=formatter)
mpl_logger = getLogger("matplotlib")  # Suppress matplotlib logging
mpl_logger.setLevel(WARNING)

## Handle logging to both logging and stdout

In [5]:
getLogger().addHandler(StreamHandler(sys.stdout))

logger = getLogger(__name__)
logger.setLevel(DEBUG)
logger.debug(f"{LOG_DIR}/{EXEC_TIME}.log")

X_train, X_test, y_train, y_test, label2act, act2label = load_raw_data()
logger.debug(f"{X_train.shape=} {X_test.shape=}")
logger.debug(f"{y_train.shape=} {y_test.shape=}")

check_class_balance(y_train.flatten(), y_test.flatten(), label2act=label2act)


/home/tahoora/Projects/college_project/Human_Activity_Recognition/notebooks/../logs/stacked_lstm_logs/deep-conv-lstm-20230725-120743/deep-conv-lstm-20230725-120743.log
X_train.shape=(7406, 128, 6, 1) X_test.shape=(2993, 128, 6, 1)
y_train.shape=(7406, 1) y_test.shape=(2993, 1)
train labels
LAYING (0): 1412 samples (19.07 %)
WALKING (1): 1224 samples (16.53 %)
WALKING_UPSTAIRS (2): 1072 samples (14.47 %)
WALKING_DOWNSTAIRS (3): 987 samples (13.33 %)
SITTING (4): 1290 samples (17.42 %)
STANDING (5): 1421 samples (19.19 %)
test labels
LAYING (0): 545 samples (18.21 %)
WALKING (1): 496 samples (16.57 %)
WALKING_UPSTAIRS (2): 470 samples (15.7 %)
WALKING_DOWNSTAIRS (3): 419 samples (14.0 %)
SITTING (4): 507 samples (16.94 %)
STANDING (5): 556 samples (18.58 %)


### Split data by preserving the percentage of samples for each class.

In [6]:
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=71)
valid_preds = np.zeros((X_train.shape[0], 6))
test_preds = np.zeros((n_splits, X_test.shape[0], 6))
models = []
scores: Dict[str, Dict[str, List[Any]]] = {
    "logloss": {"train": [], "valid": [], "test": []},
    "accuracy": {"train": [], "valid": [], "test": []},
    "precision": {"train": [], "valid": [], "test": []},
    "recall": {"train": [], "valid": [], "test": []},
    "f1": {"train": [], "valid": [], "test": []},
    "cm": {"train": [], "valid": [], "test": []},
    "per_class_f1": {"train": [], "valid": [], "test": []},
}

### Load hyper-parameters

In [7]:
with open(os.path.join(CUR_DIR, "../configs/default.json"), "r") as f:
    dcl_params = json.load(f)["stacked_lstm_params"]
    logger.debug(f"{dcl_params=}")


dcl_params={'batch_size': 128, 'epochs': 10000, 'lr': 0.001, 'verbose': 0}


In [8]:
y_test = keras.utils.to_categorical(y_test, 6)

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train[train_index, :]
    X_val = X_train[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    y_tr = keras.utils.to_categorical(y_tr, 6)
    y_val = keras.utils.to_categorical(y_val, 6)

    logger.debug(f"{X_tr.shape=} {X_val.shape=} {X_test.shape=}")
    logger.debug(f"{y_tr.shape=} {y_val.shape=} {y_test.shape=}")

    pred_tr, pred_val, pred_test, model = train_and_predict(
        LOG_DIR, fold_id, X_tr, X_val, X_test, y_tr, y_val, dcl_params
    )
    models.append(model)

    valid_preds[valid_index] = pred_val
    test_preds[fold_id] = pred_test

    for pred, X, y, mode in zip(
        [pred_tr, pred_val, pred_test], [X_tr, X_val, X_test], [y_tr, y_val, y_test], ["train", "valid", "test"]
    ):
        loss, acc = model.evaluate(X, y, verbose=0)
        pred = pred.argmax(axis=1)
        y = y.argmax(axis=1)
        scores["logloss"][mode].append(loss)
        scores["accuracy"][mode].append(acc)
        scores["precision"][mode].append(precision_score(y, pred, average="macro"))
        scores["recall"][mode].append(recall_score(y, pred, average="macro"))
        scores["f1"][mode].append(f1_score(y, pred, average="macro"))
        scores["cm"][mode].append(confusion_matrix(y, pred, normalize="true"))
        scores["per_class_f1"][mode].append(f1_score(y, pred, average=None))

X_tr.shape=(5924, 128, 6, 1) X_val.shape=(1482, 128, 6, 1) X_test.shape=(2993, 128, 6, 1)
y_tr.shape=(5924, 6) y_val.shape=(1482, 6) y_test.shape=(2993, 6)


2023-07-25 12:08:07.457454: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-25 12:08:07.789541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-25 12:08:07.790541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


2023-07-25 12:08:10.610348: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-25 12:08:10.613082: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-25 12:08:10.615046: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Creating converter from 5 to 3
Epoch 10/10000 - loss: 0.1719 - accuracy: 0.9448 - val_loss: 0.1324 - val_accuracy: 0.9521
Epoch 20/10000 - loss: 0.1557 - accuracy: 0.944 - val_loss: 0.1275 - val_accuracy: 0.9561
Epoch 30/10000 - loss: 0.1022 - accuracy: 0.9541 - val_loss: 0.0937 - val_accuracy: 0.9649
Epoch 40/10000 - loss: 0.098 - accuracy: 0.9541 - val_loss: 0.087 - val_accuracy: 0.9676
Epoch 50/10000 - loss: 0.0939 - accuracy: 0.9563 - val_loss: 0.0694 - val_accuracy: 0.9669
Epoch 60/10000 - loss: 0.094 - accuracy: 0.957 - val_loss: 0.0837 - val_accuracy: 0.9663
Epoch 70/10000 - loss: 0.0812 - accuracy: 0.9571 - val_loss: 0.0714 - val_accuracy: 0.969
Epoch 80/10000 - loss: 0.084 - accuracy: 0.9568 - val_loss: 0.0814 - val_accuracy: 0.9642
Epoch 84: early stopping
Creating converter from 3 to 5


...

2023-07-25 12:17:26.987795: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-25 12:17:26.989306: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-25 12:17:26.990597: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 10/10000 - loss: 0.1478 - accuracy: 0.9463 - val_loss: 0.1506 - val_accuracy: 0.9413
Epoch 20/10000 - loss: 0.086 - accuracy: 0.9615 - val_loss: 0.0798 - val_accuracy: 0.9581
Epoch 30/10000 - loss: 0.1152 - accuracy: 0.9571 - val_loss: 0.111 - val_accuracy: 0.9575
Epoch 40/10000 - loss: 0.0939 - accuracy: 0.961 - val_loss: 0.0998 - val_accuracy: 0.9581
Epoch 50/10000 - loss: 0.0797 - accuracy: 0.96 - val_loss: 0.0817 - val_accuracy: 0.9554
Epoch 50: early stopping


2023-07-25 12:18:24.452937: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-25 12:18:24.454734: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-25 12:18:24.455994: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

94/94 [==============================] - 1s 9ms/step


2023-07-25 12:18:29.733022: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-25 12:18:29.734802: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-25 12:18:29.736089: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
model.summary()

In [ ]:
from keras import utils

utils.plot_model(model, to_file=f"results/model.png", show_shapes=True)

# Output Cross Validation Scores

In [ ]:
logger.debug("---Cross Validation Scores---")
for mode in ["train", "valid", "test"]:
    logger.debug(f"---{mode}---")
    for metric in ["logloss", "accuracy", "precision", "recall", "f1"]:
        logger.debug(f"{metric}={round(np.mean(scores[metric][mode]))}")

    class_f1_mat = scores["per_class_f1"][mode]
    class_f1_result = {}
    for class_id in range(6):
        mean_class_f1 = np.mean([class_f1_mat[i][class_id] for i in range(n_splits)])
        class_f1_result[label2act[class_id]] = mean_class_f1
    logger.debug(f"per-class f1={round(class_f1_result)}")


# Output Final Scores Averaged over Folds

In [ ]:
logger.debug("---Final Test Scores Averaged over Folds---")
test_pred = np.mean(test_preds, axis=0).argmax(axis=1)  # average over folds
# y_test = y_test.argmax(axis=1)
logger.debug(f"accuracy={accuracy_score(y_test, test_pred)}")
logger.debug(f"precision={precision_score(y_test, test_pred, average='macro')}")
logger.debug(f"recall={recall_score(y_test, test_pred, average='macro')}")
logger.debug(f"f1={f1_score(y_test, test_pred, average='macro')}")
logger.debug(f"per-class f1={f1_score(y_test, test_pred, average=None)}")


# Plot comfusion matrix

In [ ]:
plot_confusion_matrix(
    cms=scores["cm"],
    labels=[
        "LAYING",
        "WALKING",
        "WALKING_UPSTAIRS",
        "WALKING_DOWNSTAIRS",
        "SITTING",
        "STANDING",
    ],
    path=f"{LOG_DIR}/comfusion_matrix.png",
)

In [ ]:
np.save(f"{LOG_DIR}/valid_oof.npy", valid_preds)
np.save(f"{LOG_DIR}/test_oof.npy", np.mean(test_preds, axis=0))  # Averaging

In [ ]:
from sklearn.metrics import accuracy_score

y_predict_train = model.predict(X_train).argmax(axis=1)
print("Train_accuracy is: ", accuracy_score(y_train.argmax(axis=1), y_predict_train))

y_predict_test = model.predict(X_test).argmax(axis=1)
print("Test accuracy is: ", accuracy_score(y_test, y_predict_test))

In [ ]:
from sklearn import metrics

y_predict_test = model.predict(X_test).argmax(axis=1)
y_predict_train = model.predict(X_train).argmax(axis=1)


print("Test loss is: ", np.sqrt(metrics.mean_squared_error(Y_test.argmax(axis=1), y_predict_test)))
print("Train loss is: ", np.sqrt(metrics.mean_squared_error(Y_train.argmax(axis=1), y_predict_train)))